# Single-Graph Motif Analysis: `capital-state-dallas`

Full pipeline proof of concept on one attribution graph from the Anthropic circuit-tracing paper.

**Prompt**: *"Fact: the capital of the state containing Dallas is"*  
**Task category**: Multi-hop reasoning (Dallas → Texas → Austin)

This notebook walks through:
1. Loading and exploring the graph
2. Running the triad census
3. Generating a null model ensemble and computing Z-scores
4. Identifying enriched/anti-enriched motifs
5. Visualizing the motif profile

In [ ]:
import sys
sys.path.insert(0, "..")

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

from src.graph_loader import load_attribution_graph, graph_summary
from src.motif_census import (
    compute_motif_census, enriched_motifs, motif_frequencies,
    TRIAD_LABELS, CONNECTED_TRIAD_INDICES,
    MOTIF_FAN_IN, MOTIF_FAN_OUT, MOTIF_CHAIN, MOTIF_FFL, MOTIF_CYCLE, MOTIF_COMPLETE,
)
from src.null_model import generate_configuration_null, generate_erdos_renyi_null
from src.visualization import plot_zscore_bar

sns.set_theme(style="whitegrid", font_scale=1.1)
%matplotlib inline

## 1. Load and Explore the Graph

In [ ]:
g = load_attribution_graph("../data/raw/multihop/capital-state-dallas.json")

summary = graph_summary(g)
for key, val in summary.items():
    if key not in ("layer_counts",):
        print(f"{key}: {val}")

In [ ]:
# Degree distributions
in_deg = g.indegree()
out_deg = g.outdegree()

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(in_deg, bins=30, color="steelblue", edgecolor="black", linewidth=0.5)
axes[0].set_xlabel("In-degree")
axes[0].set_ylabel("Count")
axes[0].set_title("In-Degree Distribution")
axes[0].axvline(np.mean(in_deg), color="red", linestyle="--", label=f"Mean={np.mean(in_deg):.1f}")
axes[0].legend()

axes[1].hist(out_deg, bins=30, color="darkorange", edgecolor="black", linewidth=0.5)
axes[1].set_xlabel("Out-degree")
axes[1].set_ylabel("Count")
axes[1].set_title("Out-Degree Distribution")
axes[1].axvline(np.mean(out_deg), color="red", linestyle="--", label=f"Mean={np.mean(out_deg):.1f}")
axes[1].legend()

plt.suptitle("capital-state-dallas: Degree Distributions", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Edge weight distribution
weights = g.es["weight"]
raw_weights = g.es["raw_weight"]

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(raw_weights, bins=50, color="mediumpurple", edgecolor="black", linewidth=0.5)
axes[0].set_xlabel("Raw Weight (signed)")
axes[0].set_ylabel("Count")
axes[0].set_title("Edge Weight Distribution (signed)")
axes[0].axvline(0, color="black", linewidth=0.8)

# Sign breakdown
signs = g.es["sign"]
n_exc = sum(1 for s in signs if s == "excitatory")
n_inh = sum(1 for s in signs if s == "inhibitory")
axes[1].bar(["Excitatory", "Inhibitory"], [n_exc, n_inh],
            color=["#2ca02c", "#d62728"], edgecolor="black", linewidth=0.5)
axes[1].set_ylabel("Count")
axes[1].set_title(f"Edge Signs ({n_exc} exc / {n_inh} inh)")

plt.suptitle("capital-state-dallas: Edge Properties", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Layer distribution of nodes
layers = g.vs["layer"]
feature_types = g.vs["feature_type"]

# Separate transcoder features from embedding/logit
tc_layers = [l for l, ft in zip(layers, feature_types) if ft == "cross layer transcoder"]

fig, ax = plt.subplots(figsize=(10, 4))
ax.hist(tc_layers, bins=range(min(tc_layers), max(tc_layers) + 2),
        color="teal", edgecolor="black", linewidth=0.5, align="left")
ax.set_xlabel("Layer")
ax.set_ylabel("Number of Transcoder Features")
ax.set_title("capital-state-dallas: Feature Distribution Across Layers")
plt.tight_layout()
plt.show()

## 2. Triad Census (Raw Motif Counts)

In [ ]:
census = compute_motif_census(g, size=3)

print(f"Graph: {g.vcount()} nodes, {g.ecount()} edges")
print(f"Motif size: {census.size}, classes: {census.n_classes}")
print()

# Show connected triads
connected = census.connected_counts()
total_connected = sum(connected.values())
print(f"Total connected triads: {total_connected:,}")
print()
print("Connected triad counts:")
for label, count in sorted(connected.items(), key=lambda x: -x[1]):
    pct = 100 * count / total_connected if total_connected > 0 else 0
    print(f"  {label:6s}: {count:>8,}  ({pct:5.1f}%)")

In [ ]:
# Visualize raw triad counts
conn_labels = [TRIAD_LABELS[i] for i in CONNECTED_TRIAD_INDICES]
conn_counts = [census.raw_counts[i] for i in CONNECTED_TRIAD_INDICES]

fig, ax = plt.subplots(figsize=(12, 5))
bars = ax.bar(range(len(conn_labels)), conn_counts, color="steelblue",
              edgecolor="black", linewidth=0.5)

# Highlight key motifs
key_motifs = {MOTIF_FAN_IN, MOTIF_FAN_OUT, MOTIF_CHAIN, MOTIF_FFL, MOTIF_CYCLE}
for i, idx in enumerate(CONNECTED_TRIAD_INDICES):
    if idx in key_motifs:
        bars[i].set_color("darkorange")

ax.set_xticks(range(len(conn_labels)))
ax.set_xticklabels(conn_labels, rotation=45, ha="right", fontsize=9)
ax.set_ylabel("Count")
ax.set_title("Raw Triad Counts (orange = key motif types)")
plt.tight_layout()
plt.show()

## 3. Null Model and Z-Scores

We compare the real motif counts against a **configuration model** null ensemble:
1000 degree-preserving random rewirings of the graph. For each motif class,
the Z-score tells us how many standard deviations the real count is from the
null expectation.

- **Z > 2**: Enriched motif (appears more than expected)
- **Z < -2**: Anti-enriched (appears less than expected)

In [ ]:
# Run configuration model null (1000 random graphs)
config_result = generate_configuration_null(
    g, n_random=1000, motif_size=3, show_progress=True
)

In [ ]:
# Summary table of Z-scores
print(f"{'Triad':>6s}  {'Real':>8s}  {'Null Mean':>10s}  {'Null Std':>9s}  {'Z-score':>8s}  {'SP':>7s}  Sig?")
print("-" * 72)
for i in CONNECTED_TRIAD_INDICES:
    label = TRIAD_LABELS[i]
    real = config_result.real_counts[i]
    mean = config_result.mean_null[i]
    std = config_result.std_null[i]
    z = config_result.z_scores[i]
    sp = config_result.significance_profile[i]
    sig = "***" if abs(z) > 2 else ""
    print(f"{label:>6s}  {real:>8.0f}  {mean:>10.1f}  {std:>9.1f}  {z:>8.2f}  {sp:>7.4f}  {sig}")

In [ ]:
# Z-score bar chart (main result figure)
fig = plot_zscore_bar(
    config_result,
    title='Motif Z-Score Profile: "The capital of the state containing Dallas is"',
    save_path="../figures/dallas_zscore_profile.png",
)
plt.show()

In [ ]:
# List enriched and anti-enriched motifs
sig_motifs = enriched_motifs(config_result.z_scores, threshold=2.0, labels=TRIAD_LABELS)

print("Enriched motifs:")
for m in sig_motifs:
    if m["direction"] == "enriched":
        print(f"  {m['label']:6s}  Z = {m['z_score']:+.2f}")

print("\nAnti-enriched motifs:")
for m in sig_motifs:
    if m["direction"] == "anti-enriched":
        print(f"  {m['label']:6s}  Z = {m['z_score']:+.2f}")

## 4. Significance Profile (SP)

The Significance Profile normalizes Z-scores to unit length:
$$\text{SP}_i = \frac{Z_i}{\sqrt{\sum_j Z_j^2}}$$

This makes profiles comparable across graphs of different sizes (Milo et al., 2004).

In [ ]:
sp = config_result.significance_profile

# SP bar chart
conn_sp = sp[CONNECTED_TRIAD_INDICES]
conn_labels = [TRIAD_LABELS[i] for i in CONNECTED_TRIAD_INDICES]

fig, ax = plt.subplots(figsize=(12, 5))
colors = ["#d62728" if v > 0 else "#1f77b4" for v in conn_sp]
ax.bar(range(len(conn_sp)), conn_sp, color=colors, edgecolor="black", linewidth=0.5)
ax.axhline(y=0, color="black", linewidth=0.5)
ax.set_xticks(range(len(conn_labels)))
ax.set_xticklabels(conn_labels, rotation=45, ha="right", fontsize=9)
ax.set_ylabel("SP value")
ax.set_title('Significance Profile: "The capital of the state containing Dallas is"')

plt.tight_layout()
plt.savefig("../figures/dallas_sp_profile.png", dpi=300, bbox_inches="tight")
plt.show()

print(f"SP vector norm: {np.linalg.norm(sp):.6f} (should be ~1.0)")

## 5. Null Distribution Visualization

For the two most enriched motifs (030T and 021C), plot the null distribution
and the real count to show just how extreme the enrichment is.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ax, motif_idx, color in [
    (axes[0], MOTIF_FFL, "#d62728"),
    (axes[1], MOTIF_CHAIN, "#ff7f0e"),
]:
    label = TRIAD_LABELS[motif_idx]
    null_vals = config_result.null_counts[:, motif_idx]
    real_val = config_result.real_counts[motif_idx]
    z = config_result.z_scores[motif_idx]

    ax.hist(null_vals, bins=40, color="lightgray", edgecolor="gray",
            linewidth=0.5, label="Null ensemble (n=1000)")
    ax.axvline(real_val, color=color, linewidth=2.5,
               label=f"Real count = {real_val:,}")
    ax.axvline(np.mean(null_vals), color="black", linestyle="--",
               linewidth=1, label=f"Null mean = {np.mean(null_vals):,.0f}")

    ax.set_xlabel("Motif count")
    ax.set_ylabel("Frequency")
    ax.set_title(f"{label} (Z = {z:.1f})")
    ax.legend(fontsize=9)

plt.suptitle("Real vs. Null Distribution for Enriched Motifs", fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig("../figures/dallas_null_distributions.png", dpi=300, bbox_inches="tight")
plt.show()

## 6. Secondary Null Model: Erdos-Renyi

As a robustness check, we also compare against Erdos-Renyi random graphs
(same node/edge count, but **not** preserving degree distribution).
If motifs are enriched relative to both null models, the result is more robust.

In [ ]:
er_result = generate_erdos_renyi_null(
    g, n_random=1000, motif_size=3, show_progress=True
)

In [ ]:
# Compare Z-scores from both null models
fig, ax = plt.subplots(figsize=(12, 6))

conn_z_config = config_result.z_scores[CONNECTED_TRIAD_INDICES]
conn_z_er = er_result.z_scores[CONNECTED_TRIAD_INDICES]
conn_labels = [TRIAD_LABELS[i] for i in CONNECTED_TRIAD_INDICES]

x = np.arange(len(conn_labels))
width = 0.35

ax.bar(x - width/2, conn_z_config, width, label="Configuration model",
       color="steelblue", edgecolor="black", linewidth=0.5)
ax.bar(x + width/2, conn_z_er, width, label="Erdos-Renyi",
       color="darkorange", edgecolor="black", linewidth=0.5)

ax.axhline(y=2.0, color="red", linestyle="--", alpha=0.5)
ax.axhline(y=-2.0, color="red", linestyle="--", alpha=0.5)
ax.axhline(y=0, color="black", linewidth=0.5)

ax.set_xticks(x)
ax.set_xticklabels(conn_labels, rotation=45, ha="right", fontsize=9)
ax.set_ylabel("Z-score")
ax.set_title("Z-Score Comparison: Configuration Model vs. Erdos-Renyi")
ax.legend()

plt.tight_layout()
plt.savefig("../figures/dallas_null_comparison.png", dpi=300, bbox_inches="tight")
plt.show()

## 7. Sensitivity Analysis: Edge Weight Threshold

We vary the edge weight threshold and check whether the enrichment pattern
is robust. This is critical -- if results flip at slightly different thresholds,
they aren't reliable.

In [ ]:
thresholds = [0.0, 0.5, 1.0, 2.0, 5.0]
key_motif_indices = [MOTIF_FFL, MOTIF_CHAIN, MOTIF_FAN_OUT, MOTIF_FAN_IN]
key_motif_labels = [TRIAD_LABELS[i] for i in key_motif_indices]

threshold_results = {}
for thresh in thresholds:
    g_t = load_attribution_graph(
        "../data/raw/multihop/capital-state-dallas.json",
        weight_threshold=thresh,
    )
    print(f"Threshold={thresh}: {g_t.vcount()} nodes, {g_t.ecount()} edges")
    if g_t.ecount() > 0 and g_t.vcount() >= 3:
        result = generate_configuration_null(
            g_t, n_random=200, motif_size=3, show_progress=False
        )
        threshold_results[thresh] = result
    else:
        print(f"  Skipping (too few edges)")

In [ ]:
# Plot key motif Z-scores across thresholds
fig, ax = plt.subplots(figsize=(10, 6))

colors = ["#d62728", "#ff7f0e", "#2ca02c", "#1f77b4"]
for motif_idx, label, color in zip(key_motif_indices, key_motif_labels, colors):
    threshs = sorted(threshold_results.keys())
    z_vals = [threshold_results[t].z_scores[motif_idx] for t in threshs]
    ax.plot(threshs, z_vals, "o-", color=color, label=label, linewidth=2, markersize=8)

ax.axhline(y=2.0, color="red", linestyle="--", alpha=0.4, label="Z = +/- 2")
ax.axhline(y=-2.0, color="red", linestyle="--", alpha=0.4)
ax.axhline(y=0, color="black", linewidth=0.5)

ax.set_xlabel("Edge Weight Threshold")
ax.set_ylabel("Z-score")
ax.set_title("Sensitivity Analysis: Key Motif Z-Scores vs. Edge Threshold")
ax.legend()

plt.tight_layout()
plt.savefig("../figures/dallas_sensitivity.png", dpi=300, bbox_inches="tight")
plt.show()

## 8. Size-4 Motif Census (Secondary)

Size-4 motifs have 218 isomorphism classes -- more discriminating but harder to interpret.
Let's run a quick census to see if there's additional signal.

In [ ]:
census_4 = compute_motif_census(g, size=4)
print(f"Size-4 motif classes: {census_4.n_classes}")
print(f"Non-zero classes: {sum(1 for c in census_4.raw_counts if c > 0)}")
print(f"Total size-4 motifs: {sum(c for c in census_4.raw_counts if c > 0):,}")

# Top 10 most common
sorted_4 = sorted(enumerate(census_4.raw_counts), key=lambda x: -x[1])
print("\nTop 10 most common size-4 motif classes:")
for idx, count in sorted_4[:10]:
    if count > 0:
        print(f"  Class {idx:3d}: {count:>10,}")

## 9. Summary of Findings

### Key Results for `capital-state-dallas` (Multi-Hop Reasoning)

**Enriched motifs:**
- **030T (Feedforward Loop)**: Strongly enriched. The model uses convergent evidence architectures where a feature influences the output both directly and through an intermediary.
- **021C (Chain)**: Strongly enriched. Sequential processing chains -- consistent with the multi-hop nature of the task (Dallas -> Texas -> Austin).

**Anti-enriched motifs:**
- All motifs involving **mutual edges** (102, 111D, 111U, 120U, 120C, 120D, 201, 210, 300) are anti-enriched. This confirms the architecture is fundamentally **feedforward** -- information flows in one direction through the circuit.
- **021D (Fan-out)** is anti-enriched relative to the configuration model, meaning hub-broadcasting patterns are less common than degree structure alone would predict.
- **030C (Cycle)** is anti-enriched, confirming the absence of feedback loops.

**Robustness:**
- Results are consistent across both null models (configuration and Erdos-Renyi).
- Sensitivity analysis across edge weight thresholds should confirm whether the pattern is stable.